# Natural Language Processing

Here, we will work with 'Real or Not? NLP with Disaster Tweets' data base using NLP. This data have keywords, location, target and text of some tweets. The challenge is predict if some of the 'test' tweets talks or not about disasters.

On this notebook i show the methods what i used to get 0.79252 score.

**If you liked this notebook or found something useful in it, please give it a upvote!**

**If you have some ideas to improve the notebook, please, tell me on the coments.**



## **Imports**

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
 
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# The data start divided in two parts, train and test
# The test data is what we want predict, but to do this we have to use the train data
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
# Data Length (Rows, Columns)
train.shape, test.shape

## Data cleaning

Here we will make a data cleaning, On that case, this means verify the missing values and treat the tweet text. 

In [ ]:
data = pd.concat([train, test], axis=0) # Concatenate the data 

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

Here we see all the missing values. 
So let's modify them(except the target for now).

In [ ]:
# Change all the strings to small letters 
# That makes easier to treat the data
data['text'] = data['text'].str.lower()
data['keyword'] = data['keyword'].str.lower()
data['location'] = data['location'].str.lower()

In [ ]:
# Setting the index of the columns iqual the row id
data.set_index('id', inplace=True)

Now that is a important part on this type of prediction.
Here we use a lambda function to take out some special characters 

In [ ]:
data['text'] = data['text'].apply(lambda x: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "",x))

### STOPWORD
This function removes from the data some words that harm the model
Exemple: is, he, has...


In [ ]:
def remove_words(data, col):
    stop = stopwords.words('english')
    list_of_lists = data[col].str.split()
    for idx, _ in data.iterrows():
        data[col].at[idx] = [word for word in list_of_lists[idx] \
                             if word not in stop]

In [ ]:
remove_words(data,'text')

### Lemmatizer
This function removes from the data some words that harm the model
Exemple: is, he, has...

In [ ]:
def get_word_variation(data, col):
    lemmatizer = WordNetLemmatizer()
    for idx, _ in data.iterrows():
        data[col].at[idx] = [lemmatizer.lemmatize(palavra,'v') \
                             for palavra in data[col][idx]]

In [ ]:
get_word_variation(data, 'text')

In [ ]:
data.loc[data['keyword'].notnull()].head(10)

Visualizing this data you can see some location and keywords on the text. So, lets do that with nan loaction and keywords.

In [ ]:
uniq_keyword = list(data['keyword'].unique())
uniq_location = list(data['location'].unique())

In [ ]:
for i in range(len(data)):
    if data['keyword'].isnull()[i]:
        for n in data['text'][i]:
            if n in uniq_keyword:
                data['keyword'][i] = n

In [ ]:
data.isnull().sum()

In [ ]:
for i in range(len(data)):
    if data['location'].isnull()[i]:
        for n in data['text'][i]:
            if n in uniq_location:
                data['location'][i] = n

In [ ]:
data.isnull().sum()

In [ ]:
data['location'].unique()

In [ ]:
data

In [ ]:
data['text'] = data['text'].apply(lambda x: ' '.join(x))

In [ ]:
data.isnull().sum()

In [ ]:
data.loc[data['keyword'].isnull()]

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
data['keyword'] = data['keyword'].fillna("None")
data['location'] = data['location'].fillna("None")
#data['words'] = data['words'].fillna("None")


#data.drop(['keyword', 'location', 'words'], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
train = data.loc[data['target'].notnull()]
train

In [ ]:
test = data.loc[data['target'].isnull()]
test.drop('target', axis=1, inplace=True)
test

In [ ]:
train.shape, test.shape

In [ ]:
X = train['text']
y = train['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
sgd = Pipeline([
    ('countVector', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('modelo', SGDClassifier())
])

In [ ]:
sgd.fit(X_train, y_train)

In [ ]:
sgd_pred = sgd.predict(X_test)

In [ ]:
sgd_score = f1_score(y_test, sgd_pred)

In [ ]:
sgd_score

In [ ]:
pred = sgd.predict(test['text'])

In [ ]:
test

In [ ]:
submission = pd.DataFrame({'id': test.index, 'target': pred})

In [ ]:
submission['target'] = submission['target'].astype('int')

In [ ]:
submission.to_csv('submission.csv', index=False)